# NetPyNE Tutorial 1: Simulating a simple network

In this tutorial, we will create a simple network model we can simulate. The model will consist of 40 pyramidal-like, two-compartment neurons with standard Hodgkin-Huxley dynamics in the somas and passive dynamics in the dendrites. We will connect the neurons randomly with a 10% probability of connection using a standard double-exponential synapse model. Finally, we will add a current clamp stimulus to one cell to activate the network. Then we will explore the model.

## Preliminaries

If you are going to run this notebook locally using Jupyter Notebook, start from following instructions https://github.com/suny-downstate-medical-center/netpyne/blob/development/netpyne/tutorials/README.md.

If you are using Open Source Brain or EBRAINS, everything is already set up.

On any other online platform (e.g. on Google Collab) you might need to run the following commmands to install NEURON and NetPyNE using **pip**:
```
!pip install neuron
!pip install netpyne
```

Now we are ready to start the tutorial.

## Instantiate network parameters and simulation configuration

You need two things to define a model/simulation in NetPyNE: 1) the parameters of the network and all its components (**netParams**) and 2) the configuration of the simulation (**simConfig**).  These requirements exist as objects in NetPyNE.  Let's instantiate them now.

In [ ]:
from netpyne import specs, sim
netParams = specs.NetParams()
simConfig = specs.SimConfig()

Now we are going to specify our model: a simple network of 40 pyramidal-like two-compartment neurons with standard Hodgkin-Huxley dynamics in the soma and passive dynamics in the dendrite.

## Specify a cell model

First we will add a cell type to our model by adding a dictionary named **pyr** to the *Cell Parameters* dictionary (**cellParams**) in the *Network Parameters* dictionary (**netParams**).  We will then add an empty dictionary named **secs** to hold our compartments.

In [ ]:
netParams.cellParams['pyr'] = {}
netParams.cellParams['pyr']['secs'] = {}

### Specify the soma compartment properties


Now we will define our **soma**, by adding a **geom** dictionary defining the geometry of the soma and a **mechs** dictionary defining the biophysical mechanics being added to the soma.

In [ ]:
netParams.cellParams['pyr']['secs']['soma'] = {}

In [ ]:
netParams.cellParams['pyr']['secs']['soma']['geom'] = {
    "diam": 12,
    "L": 12,
    "Ra": 100.0,
    "cm": 1
    }

In [ ]:
netParams.cellParams['pyr']['secs']['soma']['mechs'] = {"hh": {
    "gnabar": 0.12,
    "gkbar": 0.036,
    "gl": 0.0003,
    "el": -54.3
    }}

The **hh** mechanism is builtin to NEURON, but you can see its *.mod* file here:
https://github.com/neuronsimulator/nrn/blob/master/src/nrnoc/hh.mod

It is the original Hodgkin-Huxley treatment for the set of sodium, potassium, and leakage channels found in the squid giant axon membrane.

### Specify the dendrite compartment properties


Next will do the same thing for the dendrite compartment, but we will do it slightly differently.  We will first build up a **dend** dictionary and then add it to the cell model dictionary **pyr** when we are done.

In [ ]:
dend = {}

In [ ]:
dend['geom'] = {"diam": 1.0,
                "L": 200.0,
                "Ra": 100.0,
                "cm": 1,
               }

In [ ]:
dend['mechs'] = {"pas": 
                    {"g": 0.001,
                     "e": -70}
                }

The **pas** mechanim is a simple leakage channel and is builtin to NEURON.  Its *.mod* file is available here:
https://github.com/neuronsimulator/nrn/blob/master/src/nrnoc/passive.mod

In order to connect the dendrite compartment to the soma compartment, we must add a **topol** dictionary to our **dend** dictionary.

In [ ]:
dend['topol'] = {"parentSec": "soma",
                 "parentX": 1.0,
                 "childX": 0,
                }

With our **dend** section dictionary complete, we must now add it to the **pyr** cell dictionary.

In [ ]:
netParams.cellParams['pyr']['secs']['dend'] = dend

Our two-compartment cell model is now completely specified.  Our next step is to create a population of these cells.

## Specify a population of cells


NetPyNE uses *populations* of cells to specify connectivity.  In this tutorial, we will create just one population which we will call **E** (for excitatory).  It will be made of the **pyr** cells we just specified, and we want 40 of them.

In [ ]:
netParams.popParams['E'] = {
    "cellType": "pyr",
    "numCells": 40,
}

## Specify a synaptic model


We need a synaptic mechanism to connect our cells with.  We will create one called **exc** by adding a dictionary to the *synaptic mechanism parameters* dictionary (**synMechParams**).  The synapse *mod* used (**Exp2Syn**) is a simple double-exponential which is builtin to NEURON.  It's *.mod* file is available here:
https://github.com/neuronsimulator/nrn/blob/master/src/nrnoc/exp2syn.mod

In [ ]:
netParams.synMechParams['exc'] = {
    "mod": "Exp2Syn",
    "tau1": 0.1,
    "tau2": 1.0,
    "e": 0
}

## Specify the connectivity


Now we will specify the connectivity in our model by adding an entry to the **connParams** dictionary.  We will call our connectivity rule **E->E** as it will define connectivity from our **E** population to our **E** population.

We will use the *synMech* **exc**, which we defined above.  For this synaptic mechanism, a *weight* of about **0.005** is appropriate.  These cells will have a 10% probability of getting connected, and will be activated five milliseconds after an action potential occurs in the presynaptic cell.  Synapses will occur on the **dend** *section* at its very end (*location* **1.0**)

In [ ]:
netParams.connParams['E->E'] = {
    "preConds": {"pop": "E"},
    "postConds": {"pop": "E"},
    "weight": 0.005,
    "probability": 0.1,
    "delay": 5.0,
    "synMech": "exc",
    "sec": "dend",
    "loc": 1.0,
}

## Set up the simulation configuration

In [ ]:
simConfig.filename = "netpyne_tut1"
simConfig.duration = 200.0
simConfig.dt = 0.1

We will record from from the first cell (**0**) and we will record the voltage in the middle of the soma and the end of the dendrite.

In [ ]:
simConfig.recordCells = [0]
simConfig.recordTraces = {
    "V_soma": {
        "sec": "soma",
        "loc": 0.5,
        "var": "v",
    },
    "V_dend": {
        "sec": "dend",
        "loc": 1.0,
        "var": "v",
    }
}

Finally we will set up some plots to be automatically generated and saved.

In [ ]:
simConfig.analysis = {
    "plotTraces": {
        "include": [0],
        "saveFig": True,
    },
    "plotRaster": {
        "saveFig": True,
    }
}

## Create, simulate, and analyze the model


The command `%matplotlib inline` allows figures to be shown in this notebook.

In [ ]:
%matplotlib inline

Use one simple command to create, simulate, and analyze the model.

In [ ]:
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)

We can see that there was no spiking in the network, and thus the spike raster was not plotted.  But there should be one new file in your directory: **netpyne_tut1_traces.png**.  Take a look.  Not too interesting, the cell just settles into its resting membrane potential.

Let's overlay the traces.  You can see all the options available for **plotTraces** here: http://netpyne.org/netpyne.analysis.traces.html#netpyne.analysis.traces.plotTraces


In [ ]:
sim.analysis.plotTraces(overlay=True);

## Explore the model

### Plot the 2D connectivity of the network

Now we can take a look at the physical layout of our network model.  You can see all the options available for **plot2Dnet** here:
http://netpyne.org/netpyne.analysis.network.html#netpyne.analysis.network.plot2Dnet

In [ ]:
sim.analysis.plot2Dnet();

### Plot the connectivity matrix


You can see all the options available for **plotConn** here:
http://netpyne.org/netpyne.analysis.network.html#netpyne.analysis.network.plotConn

In [ ]:
sim.analysis.plotConn();

Not very interesting with just one population, but we can also look at the cellular level connectivity.

In [ ]:
sim.analysis.plotConn(feature='weight', groupBy='cell');

## Add a stimulation


We'll need to kickstart this network to see some activity -- let's inject current into one of the cells.  First we need to add an entry to the *Stimulation Source Parameters* dictionary (**stimSourceParams**).  We'll call our stimulation **IClamp1**, and we'll use the standard NEURON *type*: **IClamp**.  The current injection will last for a *duration* of 20 ms, it will start at a *delay* of 5 ms, and it will have an *amplitude* of 0.1 nanoAmps. 

In [ ]:
netParams.stimSourceParams['IClamp1'] = {
    "type": "IClamp",
    "dur": 5,
    "del": 20,
    "amp": 0.1,
}

Now we need to add a target for our stimulation.  We do that by adding a dictionary to the *Stimulation Target Parameters* dictionary (**stimTargetParams**).  We'll call this connectivity rule **IClamp1->cell0**, because it will go from the source we just created (**IClamp1**) and the first cell in our population.  The stimulation (current injection in this case) will occur in our **dend** *section* at the very tip (*location* of **1.0**).

In [ ]:
netParams.stimTargetParams['IClamp1->cell0'] = {
    "source": "IClamp1",
    "conds": {"cellList": [0]},
    "sec": "dend",
    "loc": 1.0,
}

Now we can re-run the simulation.

In [ ]:
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)

Now we see spiking in the network, and the raster plot appears.  Let's improve the plots a little bit.

In [ ]:
sim.analysis.plotTraces(overlay=True);

In [ ]:
sim.analysis.plotRaster(marker='o', markerSize=50);

## Record and plot a variety of traces


Now let's explore the model by recording and plotting a variety of traces.  First let's clear our **recordTraces** dictionary and turn off the automatic raster plot.

In [ ]:
simConfig.recordTraces = {}
simConfig.analysis['plotRaster'] = False

### Record and plot the somatic conductances


Let's record and plot the somatic conductances.  We need to take a look at the **hh** mod file to see what the variables are called.  The file is available here: https://github.com/neuronsimulator/nrn/blob/master/src/nrnoc/hh.mod

We can see that the conductances are called *gna*, *gk*, and *gl*.  Let's set up recording for these conductances in the middle of the soma.

In [ ]:
simConfig.recordTraces['gNa'] = {'sec': 'soma', 'loc': 0.5, 'mech': 'hh', 'var': 'gna'}
simConfig.recordTraces['gK'] = {'sec': 'soma', 'loc': 0.5, 'mech': 'hh', 'var': 'gk'}
simConfig.recordTraces['gL'] = {'sec': 'soma', 'loc': 0.5, 'mech': 'hh', 'var': 'gl'}

Then we can re-run the simulation.

In [ ]:
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)

Let's zoom in on one spike and overylay the traces.

In [ ]:
sim.analysis.plotTraces(timeRange=[90, 110], overlay=True);

### Record from synapses

Our synapses are set up to use **Exp2Syn**, which is builtin to NEURON.  Its mod file is available here: https://github.com/neuronsimulator/nrn/blob/master/src/nrnoc/exp2syn.mod

Looking in the file, we can see that its current variable is called **i**.  Let's record that and the voltages in the dendrite and soma.

In [ ]:
simConfig.recordTraces = {}
simConfig.recordTraces['iSyn0'] = {'sec': 'dend', 'loc': 1.0, 'synMech': 'exc', 'var': 'i'}
simConfig.recordTraces['V_dend'] = {'sec': 'dend', 'loc': 1.0, 'var': 'v'}
simConfig.recordTraces['V_soma'] = {'sec': 'soma', 'loc': 0.5, 'var': 'v'}

In [ ]:
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)

That's the first synapse created in that location, but there are likely multiple synapses.  Let's plot all the synaptic currents entering cell 0.  First we need to see what they are.  The network is defined in **sim.net**.  Type in *sim.net.* and then push *Tab* to see what's available.

The data for cell 0 is in **sim.net.allCells[0]**.

In [ ]:
sim.net.allCells[0].keys()

The connections coming onto the cell are in the **conns** dictionary.

In [ ]:
sim.net.allCells[0]['conns']

So we want to record six synaptic currents.  Lets do that in a *for loop* at the same time creating a dictionary to hold the synaptic trace names as keys (and later the trace arrays as values).

In [ ]:
simConfig.recordTraces = {}
simConfig.recordTraces['V_soma'] = {'sec': 'soma', 'loc': 0.5, 'var': 'v'}
simConfig.recordTraces['V_dend'] = {'sec': 'dend', 'loc': 1.0, 'var': 'v'}

syn_plots = {}
for index, presyn in enumerate(sim.net.allCells[0]['conns']):    
    trace_name = 'i_syn_' + str(presyn['preGid'])
    syn_plots[trace_name] = None 
    simConfig.recordTraces[trace_name] = {'sec': 'dend', 'loc': 1.0, 'synMech': 'exc', 'var': 'i', 'index': index}

Let's take a look at our **recordTraces** dictionary now.

In [ ]:
simConfig.recordTraces

Now we'll run the simulation again so that these traces get recorded.

In [ ]:
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)

## Extracting recorded data

Let's make our synaptic currents plot nicer.  We'll make a figure with two plots, the top one will be the somatic and dendritic voltage and the bottom plot will be all of the synaptic currents overlaid.

First we'll have to extract the data.  Simulation data gets stored in the dictionary **sim.allSimData**.

In [ ]:
sim.allSimData.keys()

**spkt** is an array of the times of all spikes in the network.  **spkid** is an array of the universal index (GID) of the cell spiking.   **t** is an array of the time for traces.  Our traces appear as we named them, and each is a dictionary with its key being **cell_GID** and its value being the array of the trace.

In [ ]:
sim.allSimData.V_soma.keys()

So let's extract our data.

In [ ]:
time = sim.allSimData['t']
v_soma = sim.allSimData['V_soma']['cell_0']
v_dend = sim.allSimData['V_dend']['cell_0']

for syn_plot in syn_plots:
    syn_plots[syn_plot] = sim.allSimData[syn_plot]['cell_0']

And now we can make our custom plot.

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=[10, 8])

plt.subplot(211)
plt.plot(time, v_soma, label='v_soma')
plt.plot(time, v_dend, label='v_dend')
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Membrane potential (mV)')

plt.subplot(212)
for syn_plot in syn_plots:
    plt.plot(time, syn_plots[syn_plot], label=syn_plot)
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Synaptic current (nA)')

plt.savefig('syn_currents.jpg', dpi=600)

Congratulations!  You have simulated a network model by specifying parameters for cells, populations, synapses, connectivity, stimulations, and recording.  You explored some basic analyses and extracted data to make a custom plot.

Now we will put all of this together into a single file.

## This tutorial in a single Python file

In [ ]:
from netpyne import specs, sim
netParams = specs.NetParams()
simConfig = specs.SimConfig()

# Create a cell type
# ------------------

netParams.cellParams['pyr'] = {}
netParams.cellParams['pyr']['secs'] = {}

# Add a soma section
netParams.cellParams['pyr']['secs']['soma'] = {}
netParams.cellParams['pyr']['secs']['soma']['geom'] = {
    "diam": 12,
    "L": 12,
    "Ra": 100.0,
    "cm": 1
    }

# Add hh mechanism to soma
netParams.cellParams['pyr']['secs']['soma']['mechs'] = {"hh": {
    "gnabar": 0.12,
    "gkbar": 0.036,
    "gl": 0.0003,
    "el": -54.3
    }}

# Add a dendrite section
dend = {}
dend['geom'] = {"diam": 1.0,
                "L": 200.0,
                "Ra": 100.0,
                "cm": 1,
               }

# Add pas mechanism to dendrite
dend['mechs'] = {"pas": 
                    {"g": 0.001,
                     "e": -70}
                }

# Connect the dendrite to the soma
dend['topol'] = {"parentSec": "soma",
                 "parentX": 1.0,
                 "childX": 0,
                }

# Add the dend dictionary to the cell parameters dictionary
netParams.cellParams['pyr']['secs']['dend'] = dend

# Create a population of these cells
# ----------------------------------
netParams.popParams['E'] = {
    "cellType": "pyr",
    "numCells": 40,
}

# Add Exp2Syn synaptic mechanism
# ------------------------------
netParams.synMechParams['exc'] = {
    "mod": "Exp2Syn",
    "tau1": 0.1,
    "tau2": 1.0,
    "e": 0
}

# Define the connectivity
# -----------------------
netParams.connParams['E->E'] = {
    "preConds": {"pop": "E"},
    "postConds": {"pop": "E"},
    "weight": 0.005,
    "probability": 0.1,
    "delay": 5.0,
    "synMech": "exc",
    "sec": "dend",
    "loc": 1.0,
}

# Add a stimulation
# -----------------
netParams.stimSourceParams['IClamp1'] = {
    "type": "IClamp",
    "dur": 5,
    "del": 20,
    "amp": 0.1,
}

# Connect the stimulation
# -----------------------
netParams.stimTargetParams['IClamp1->cell0'] = {
    "source": "IClamp1",
    "conds": {"cellList": [0]},
    "sec": "dend",
    "loc": 1.0,
}

# Set up the simulation configuration
# -----------------------------------

simConfig.filename = "netpyne_tut1"
simConfig.duration = 200.0
simConfig.dt = 0.1

# Record from cell 0
simConfig.recordCells = [0]

# Record the voltage at the soma and the dendrite
simConfig.recordTraces = {
    "V_soma": {
        "sec": "soma",
        "loc": 0.5,
        "var": "v",
    },
    "V_dend": {
        "sec": "dend",
        "loc": 1.0,
        "var": "v",
    }
}

# Record somatic conductances
#simConfig.recordTraces['gNa'] = {'sec': 'soma', 'loc': 0.5, 'mech': 'hh', 'var': 'gna'}
#simConfig.recordTraces['gK'] = {'sec': 'soma', 'loc': 0.5, 'mech': 'hh', 'var': 'gk'}
#simConfig.recordTraces['gL'] = {'sec': 'soma', 'loc': 0.5, 'mech': 'hh', 'var': 'gl'}

# Automatically generate some figures
simConfig.analysis = {
    "plotTraces": {
        "include": [0],
        "saveFig": True,
        "overlay": True,
    },
    "plotRaster": {
        "saveFig": True,
        "marker": "o",
        "markerSize": 50,
    },
    "plotConn": {
        "saveFig": True,
        "feature": "weight",
        "groupby": "cell",
        "markerSize": 50,
    },
    "plot2Dnet": {
        "saveFig": True,
    },
}


# Create, simulate, and analyze the model
# ---------------------------------------
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)


# Set up the recording for the synaptic current plots
syn_plots = {}
for index, presyn in enumerate(sim.net.allCells[0]['conns']):    
    trace_name = 'i_syn_' + str(presyn['preGid'])
    syn_plots[trace_name] = None 
    simConfig.recordTraces[trace_name] = {'sec': 'dend', 'loc': 1.0, 'synMech': 'exc', 'var': 'i', 'index': index}

    
# Create, simulate, and analyze the model
# ---------------------------------------
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)
    

# Extract the data
# ----------------
time = sim.allSimData['t']
v_soma = sim.allSimData['V_soma']['cell_0']
v_dend = sim.allSimData['V_dend']['cell_0']

for syn_plot in syn_plots:
    syn_plots[syn_plot] = sim.allSimData[syn_plot]['cell_0']

    
# Plot our custom figure
# ----------------------
import matplotlib.pyplot as plt
fig = plt.figure()

plt.subplot(211)
plt.plot(time, v_soma, label='v_soma')
plt.plot(time, v_dend, label='v_dend')
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Membrane potential (mV)')

plt.subplot(212)
for syn_plot in syn_plots:
    plt.plot(time, syn_plots[syn_plot], label=syn_plot)
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Synaptic current (nA)')

plt.savefig('syn_currents.jpg', dpi=600)